In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pymysql
import tensorflow as tf
import pandas.io.sql as psql

# Mysql 원격 접속

In [115]:
con = pymysql.connect(host="18.218.142.199", user="root", passwd="111111", db="race", charset="utf8")
cursor = con.cursor()

In [116]:

ban_script = ("select * from ban limit 1000")
cure_script = ("select * from cure limit 1000")
grade_script = ("select * from grade limit 1000")
race_record_script = ("select * from race_record limit 1000")
race_test_script = ("select * from race_test limit 1000")
rider_script = ("select * from rider limit 1000")
rider_bu_script = ("select * from rider_bu limit 1000")
sudo_total_script = ("select * from sudo_total limit 1000")
total_hn_bu_script = ("select * from total_hn_bu limit 1000")
train_script = ("select * from train limit 1000")

ban = psql.read_sql(ban_script, con)
cure = psql.read_sql(cure_script, con)
grade_script = psql.read_sql(grade_script, con)
race_record = psql.read_sql(race_record_script, con)
race_test = psql.read_sql(race_test_script, con)
rider = psql.read_sql(rider_script, con)
rider_bu = psql.read_sql(rider_bu_script, con)
sudo_total = psql.read_sql(sudo_total_script, con)
total_hn_bu = psql.read_sql(total_hn_bu_script, con)
train = psql.read_sql(train_script, con)

con.close()

# sqlite3 local 접속

In [3]:
import sqlite3

In [4]:
con = sqlite3.connect('/Users/daham/Downloads/race.db')
cursor = con.cursor()

# cursor.execute("SELECT total_hn_bu.date, total_hn_bu.no, total_hn_bu.distance, total_hn_bu.horse_name, total_hn_bu.horse_no, \
#                 total_hn_bu.horse_weight, total_hn_bu.horse_weight_pm, total_hn_bu.location, total_hn_bu.race_time, \
#                 total_hn_bu.road_cd_pc, total_hn_bu.weather, total_hn_bu.weight_up, total_hn_bu.경기등급1, total_hn_bu.기수명, \
#                 total_hn_bu.단승, total_hn_bu.번호, total_hn_bu.복승식, total_hn_bu.산지, total_hn_bu.성별, total_hn_bu.순위, \
#                 total_hn_bu.연령, total_hn_bu.연승, total_hn_bu.중량, grade.등급 FROM total_hn_bu, grade \
#                 WHERE total_hn_bu.horse_weight <> 0 AND total_hn_bu.race_time <> '' AND total_hn_bu.road_cd_pc IS NOT NULL AND \
#                 total_hn_bu.weather <> '' AND total_hn_bu.복승식 <> 0 AND grade.horse_no = total_hn_bu.horse_no AND\
#                 grade.시작일자 <= total_hn_bu.date AND grade.종료일자 >= total_hn_bu.date ORDER BY total_hn_bu.horse_no, \
#                 total_hn_bu.date")
    
# total = pd.DataFrame(cursor.fetchall())

### 현세님이 준거

In [5]:
# 날짜 뽑기
cursor.execute("SELECT total_hn_bu.date from total_hn_bu, grade WHERE total_hn_bu.horse_weight <> 0 AND total_hn_bu.race_time <> '' AND total_hn_bu.road_cd_pc IS NOT NULL AND total_hn_bu.weather <> '' AND total_hn_bu.복승식 <> 0 AND grade.horse_no = total_hn_bu.horse_no AND grade.시작일자 <= total_hn_bu.date AND grade.종료일자 >= total_hn_bu.date ORDER BY total_hn_bu.horse_no, total_hn_bu.date")
date = pd.Series(cursor.fetchall())

In [6]:
date_revised = date.apply(lambda x : pd.to_datetime(x))

In [7]:
time_difference = date_revised - min(date_revised)

In [55]:
# 현세님 pickle data 받기
df = pd.read_pickle('/Users/daham/Desktop/GrowthHackers/horserace/neural-network/df.pickle')

In [56]:
df.shape

(116779, 30)

In [57]:
df = pd.DataFrame(data=df, columns=['record_time', 'round', 'distance', 'humidity', 'weight_ctrl', 'dan', 'lane', 'bok', 'age', 'yeon', 'jk_weight', 'weight', 'location', 'rating', 'race_grade', 'weather1', 'weather2', 'weather3', 'weather4', 'weather5', 'nation1', 'nation2', 'nation3', 'nation4', 'nation5', 'nation6', 'nation7', 'nation8', 'sex', 'cure_1mnth'])

In [58]:
df['date'] = time_difference.apply(lambda x : x.days[0])

In [59]:
df[['weather1', 'weather2', 'weather3', 'weather4',
    'weather5', 'nation1', 'nation2', 'nation3', 'nation4', 'nation5',
    'nation6', 'nation7', 'nation8', 'sex', 'cure_1mnth']] = df[['weather1', 'weather2', 'weather3', 'weather4',
    'weather5', 'nation1', 'nation2', 'nation3', 'nation4', 'nation5',
    'nation6', 'nation7', 'nation8', 'sex', 'cure_1mnth']].astype(int).astype(str)


In [60]:
df.head(6)

,record_time,round,distance,humidity,weight_ctrl,dan,lane,bok,age,yeon,...,nation2,nation3,nation4,nation5,nation6,nation7,nation8,sex,cure_1mnth,date
0,117.7,6.0,1800.0,3.0,0.0,99.5,12.0,94.4,10.0,10.7,...,0,0,0,0,0,0,0,1,0,22
1,130.8,8.0,2000.0,3.0,0.0,25.8,5.0,30.6,10.0,4.7,...,0,0,0,0,0,0,0,1,1,50
2,131.6,7.0,2000.0,12.0,0.0,28.6,4.0,45.1,10.0,4.3,...,0,0,0,0,0,0,0,1,1,71
3,131.6,4.0,2000.0,4.0,0.0,13.9,12.0,514.7,10.0,3.5,...,0,0,0,0,0,0,0,1,1,92
4,118.1,5.0,1800.0,4.0,0.0,83.8,3.0,43.5,10.0,12.4,...,0,0,0,0,0,0,0,1,1,120
5,116.9,6.0,1800.0,3.0,0.0,11.4,6.0,17.0,10.0,4.0,...,0,0,0,0,0,0,0,1,1,148


In [61]:
# 순위 뽑기
cursor.execute("SELECT total_hn_bu.순위 from total_hn_bu, grade WHERE total_hn_bu.horse_weight <> 0 AND total_hn_bu.race_time <> '' AND total_hn_bu.road_cd_pc IS NOT NULL AND total_hn_bu.weather <> '' AND total_hn_bu.복승식 <> 0 AND grade.horse_no = total_hn_bu.horse_no AND grade.시작일자 <= total_hn_bu.date AND grade.종료일자 >= total_hn_bu.date ORDER BY total_hn_bu.horse_no, total_hn_bu.date")
rank = pd.Series(cursor.fetchall())

In [62]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(df, rank, test_size=0.3, random_state=0)

# data 컬럼 별로 특징 확인 필요 (scaling 또는 normalization 필요한지)

In [53]:
df.columns

Index(['record_time', 'round', 'distance', 'humidity', 'weight_ctrl', 'dan',
       'lane', 'bok', 'age', 'yeon', 'jk_weight', 'weight', 'location',
       'rating', 'race_grade', 'weather1', 'weather2', 'weather3', 'weather4',
       'weather5', 'nation1', 'nation2', 'nation3', 'nation4', 'nation5',
       'nation6', 'nation7', 'nation8', 'sex', 'cure_mnth', 'date',
       'cure_1mnth'],
      dtype='object')

In [63]:
df.loc[:,'cure_1mnth'].unique()

array(['0', '1'], dtype=object)

In [64]:
# Categorical Variables (확인필요)
# round_ = tf.feature_column.categorical_column_with_vocabulary_list("round", list(map(str, np.arange(1,16))))
#
#
#
weather1 = tf.feature_column.categorical_column_with_vocabulary_list("weather1", ["0", "1"])
weather2 = tf.feature_column.categorical_column_with_vocabulary_list("weather2", ["0", "1"])
weather3 = tf.feature_column.categorical_column_with_vocabulary_list("weather3", ["0", "1"])
weather4 = tf.feature_column.categorical_column_with_vocabulary_list("weather4", ["0", "1"])
weather5 = tf.feature_column.categorical_column_with_vocabulary_list("weather5", ["0", "1"])

nation1 = tf.feature_column.categorical_column_with_vocabulary_list("nation1", ["0", "1"])
nation2 = tf.feature_column.categorical_column_with_vocabulary_list("nation2", ["0", "1"])
nation3 = tf.feature_column.categorical_column_with_vocabulary_list("nation3", ["0", "1"])
nation4 = tf.feature_column.categorical_column_with_vocabulary_list("nation4", ["0", "1"])
nation5 = tf.feature_column.categorical_column_with_vocabulary_list("nation5", ["0", "1"])
nation6 = tf.feature_column.categorical_column_with_vocabulary_list("nation6", ["0", "1"])
nation7 = tf.feature_column.categorical_column_with_vocabulary_list("nation7", ["0", "1"])
nation8 = tf.feature_column.categorical_column_with_vocabulary_list("nation8", ["0", "1"])

sex = tf.feature_column.categorical_column_with_vocabulary_list("sex", ["0", "1"])
cure_1mnth = tf.feature_column.categorical_column_with_vocabulary_list("cure_1mnth", ["0", "1"])


In [65]:
# Continuous Variables (확인필요 - 일단은 다 numerical 로..)
record_time = tf.feature_column.numeric_column("record_time")
round_ = tf.feature_column.numeric_column("round")
distance = tf.feature_column.numeric_column("distance")
humidity = tf.feature_column.numeric_column("humidity")
weight_ctrl = tf.feature_column.numeric_column("weight_ctrl")
dan = tf.feature_column.numeric_column("dan")
lane = tf.feature_column.numeric_column("lane")
bok = tf.feature_column.numeric_column("bok")
age = tf.feature_column.numeric_column("age")
yeon = tf.feature_column.numeric_column("yeon")
jk_weight = tf.feature_column.numeric_column("jk_weight")
weight = tf.feature_column.numeric_column("weight")
location = tf.feature_column.numeric_column("location")
rating = tf.feature_column.numeric_column("rating")
race_grade = tf.feature_column.numeric_column("race_grade")
date = tf.feature_column.numeric_column("date")



In [67]:
feat_cols = [record_time 
            ,round_
            ,distance
            ,humidity
            ,weight_ctrl
            ,dan
            ,lane
            ,bok
            ,age
            ,yeon
            ,jk_weight
            ,weight
            ,location
            ,rating
            ,race_grade
            ,weather1
            ,weather2
            ,weather3
            ,weather4
            ,weather5
            ,nation1
            ,nation2
            ,nation3
            ,nation4
            ,nation5
            ,nation6
            ,nation7
            ,nation8
            ,sex
            ,cure_1mnth
            ,date]

In [68]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [69]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/8q/km98z9qn4vq4g45kcsnyll9r0000gn/T/tmphn0wm7gt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x16a66f748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [70]:
sess = tf.Session()
new_saver = tf.train.import_meta_graph('../variables_pred/model.ckpt-10151.meta', clear_devices=True)
new_saver.restore(sess, tf.train.latest_checkpoint('../variables_pred/'))
all_vars = tf.trainable_variables()
for v in all_vars:
    print(v.name)
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InternalError'>, Unable to get element as bytes.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/8q/km98z9qn4vq4g45kcsnyll9r0000gn/T/tmphn0wm7gt/model.ckpt.


InternalError: Unable to get element as bytes.

In [48]:
with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (N,1)) 
    y = tf.placeholder(tf.float32, (N,))
    
with tf.name_scope("weights"):
    W = tf.Variable(tf.random_normal((1,1)))
    b = tf.Variable(tf.random_normal((1,)))
    
with tf.name_scope("prediction"):
    y_pred = tf.matmul(x, W) + b
with tf.name_scope("loss"):
    l = tf.reduce_sum((y-y_pred)**2)
    
with tf.name_scope("optim"):
    train_op = tf.train.AdamOptimizer(0.001).minimize(l)
with tf.name_scope("summaries"):
    tf.summary.scalar("loss", l)
    merged = tf.summary.merge_all()

train_writer = tf.summary.FileWriter('/Users/daham/Desktop/GrowthHackers/horserace/neural-network/lr-train', tf.get_default_graph())

# 전처리

In [127]:
# race_noheader 에서 distance 가져오기 (속력을 구하기 위해)

# primary key는 date & round

In [114]:
race_db = pd.read_csv('/Users/daham/Downloads/tables 181110/race_noheader.csv')
first_row_race_db = np.array(race_db.columns)
tmp_race_db = np.array(race_db)

In [119]:
final_race_df = pd.DataFrame(np.vstack((first_row_race_db, tmp_race_db)))
final_race_df = final_race_df[[0,1,5]]
final_race_df.columns = ['date', 'round', 'distance']

In [120]:
final_race_df

,date,round,distance
0,2018-10-07,11,1300
1,2018-10-07,10,1700
2,2018-10-07,9,2000
3,2018-10-07,8,1700
4,2018-10-07,7,1000
5,2018-10-07,6,1000
6,2018-10-07,5,1300
7,2018-10-07,4,1200
8,2018-10-07,3,1000
9,2018-10-07,2,1000


In [128]:
# to_sql_race_result 에서 모두 가져오기

# 이전 3경기의 순위를 새로운 컬럼에 기록 

# primary key는 date & round 로 해서 위의 final_race_df 와 merge

In [123]:
to_sql_race_result_df = pd.read_csv('/Users/daham/Downloads/tables 181110/to_sql_race_result.csv')
first_row_rae_result = np.array(to_sql_race_result_df.columns)
tmp_to_sql_race_result_df = np.array(to_sql_race_result_df)

In [129]:
final_to_sql_race_result_df = pd.DataFrame(np.vstack((first_row_rae_result, tmp_to_sql_race_result_df)), columns = ['url', 'date', 'round', 'name', 'code', 'rank', 'lane', 'sex', 'age', 'jockey_w', 'rating', 'jockey','difference', 'weight','dandivi', 'yeondivi', 'equipment','s1fr', 'c1r', 'c2r', 'c3r', 'c4r', 'g1fr', 's1f', 'c1', 'c2', 'c3','c4','g3f', 'record'])

In [125]:
final_df.columns


Index(['url', 'date', 'round', 'name', 'code', 'rank', 'lane', 'sex', 'age',
       'jockey_w', 'rating', 'jockey', 'difference', 'weight', 'dandivi',
       'yeondivi', 'equipment', 's1fr', 'c1r', 'c2r', 'c3r', 'c4r', 'g1fr',
       's1f', 'c1', 'c2', 'c3', 'c4', 'g3f', 'record'],
      dtype='object')

In [126]:
final_df.head()

,url,date,round,name,code,rank,lane,sex,age,jockey_w,...,c3r,c4r,g1fr,s1f,c1,c2,c3,c4,g3f,record
0,http://www.gumvit.com/statv40/result_detail.ht...,2018-11-03,6,승리나래,036805,1,8,거,3,56.0,...,4.1,3.2,13.8,Unnamed: 23,Unnamed: 24,30.9,49.8,39.2,13.6,82.6
1,http://www.gumvit.com/statv40/result_detail.ht...,2018-11-03,6,라스트잭팟,35049,2,3,거,4,53,...,6,4,14.2,NaN,NaN,31.7,50.2,38.7,13.6,82.7
2,http://www.gumvit.com/statv40/result_detail.ht...,2018-11-03,6,승리플러스,37107,3,12,수,3,52.5,...,5,5,13.8,NaN,NaN,31.2,50.1,39,13.6,82.7
3,http://www.gumvit.com/statv40/result_detail.ht...,2018-11-03,6,라온제임스,38516,4,1,수,2,54,...,1,1,13.7,NaN,NaN,30.7,49.4,39.6,14,82.7
4,http://www.gumvit.com/statv40/result_detail.ht...,2018-11-03,6,백두의정상,37583,5,2,수,3,55.5,...,2,2,13.8,NaN,NaN,30.7,49.5,40.2,14.5,83.4


In [130]:
horse_list = list(final_to_sql_race_result_df.name.unique())

In [109]:
empty_df = pd.DataFrame(columns=['date', 'name', 'rank'])
for i in horse_list:
#     final_to_sql_race_result_df.loc[final_to_sql_race_result_df.name=='힘찬출발', ['date','name','rank']].sort_values('date')
    tmp = final_to_sql_race_result_df.loc[final_to_sql_race_result_df.name==i, :].sort_values('date')
    

,url,date,round,name,code,rank,lane,sex,age,jockey_w,...,c3r,c4r,g1fr,s1f,c1,c2,c3,c4,g3f,record
19891,http://www.gumvit.com/statv40/result_detail.ht...,2017-02-25,6,힘찬출발,33585,10,7,거,4,54,...,6,9,14,NaN,NaN,31.4,50.6,41.5,14.5,85.5
22007,http://www.gumvit.com/statv40/result_detail.ht...,2016-12-18,5,힘찬출발,33585,중,10,거,3,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23003,http://www.gumvit.com/statv40/result_detail.ht...,2016-11-20,4,힘찬출발,33585,11,11,거,3,55.5,...,12,12,15.3,NaN,NaN,26.5,44.7,39.4,13.9,77.9
24667,http://www.gumvit.com/statv40/result_detail.ht...,2016-10-02,3,힘찬출발,33585,1,6,거,3,56,...,4,2,14.3,NaN,NaN,31.9,50.9,39.9,13.9,84.4
25994,http://www.gumvit.com/statv40/result_detail.ht...,2016-08-21,2,힘찬출발,33585,4,1,거,3,56,...,9,7,15,NaN,NaN,32.7,51.5,40.3,14.1,85.4
27164,http://www.gumvit.com/statv40/result_detail.ht...,2016-07-16,2,힘찬출발,33585,5,6,거,3,56,...,12,11,15.2,NaN,NaN,15.2,32.3,36.2,12.4,62.4
38697,http://www.gumvit.com/statv40/result_detail.ht...,2015-08-15,4,힘찬발걸음,33409,8,9,수,2,54,...,11,10,14.5,NaN,NaN,14.5,32.3,38.6,13.2,64.4
52788,http://www.gumvit.com/statv40/result_detail.ht...,2014-07-12,7,힘찬나래,29684,중,9,거,3,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53665,http://www.gumvit.com/statv40/result_detail.ht...,2014-06-15,3,힘찬나래,29684,3,2,거,3,54,...,1,3,13.7,NaN,NaN,31.3,51,40.2,13.8,84.5
54624,http://www.gumvit.com/statv40/result_detail.ht...,2014-05-18,2,힘찬나래,29684,7,9,거,3,54,...,1,1,13.8,NaN,NaN,31,50.5,41.5,14.6,85.3


In [152]:
final_to_sql_race_result_df.loc[final_to_sql_race_result_df.name=='힘찬출발',['date', 'name','rank']].sort_values('date', ascending=False).shift(periods=0, axis=0)


,date,name,rank
19891,2017-02-25,힘찬출발,10
22007,2016-12-18,힘찬출발,중
23003,2016-11-20,힘찬출발,11
24667,2016-10-02,힘찬출발,1
25994,2016-08-21,힘찬출발,4
27164,2016-07-16,힘찬출발,5
